In [1]:
import torch
import torchvision
import datetime
from torchvision import transforms
from torchvision.models import resnext50_32x4d
from PIL import Image
import time
import pytz

def read_classes():
    """
    Load the ImageNet class names.
    """
    with open("imagenet-classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    return categories

print("Before loading model: ",datetime.datetime.now(pytz.timezone('Europe/London')))
# Load the pre-trained ResNet model
model = resnext50_32x4d(pretrained=True)
# print(model)
categories = read_classes()
model.eval()  # Set the model to evaluation mode



Before loading model:  2024-10-18 19:00:11.910000+01:00


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [17]:
# Load and preprocess an example image
print("Loading image: ",datetime.datetime.now(pytz.timezone('Europe/London')))
image_path = 'dog_1.jpg'
input_image = Image.open(image_path).convert('RGB')
# print("Image transformation: ",jetson.stats)

Loading image:  2024-10-18 19:07:29.598854+01:00


In [18]:
print("Image transformation: ",datetime.datetime.now(pytz.timezone('Europe/London')))
# Define the image transformation pipeline
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Apply transformations to the input image
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension


Image transformation:  2024-10-18 19:07:32.692963+01:00


In [19]:
# Make the prediction
with torch.no_grad():
    start_time = time.time()
    print("During prediction: ", datetime.datetime.now(pytz.timezone('Europe/London')))
    # print("During prediction: ",jtop().stats)
    output = model(input_batch)
    end_time = time.time()

# Get the predicted class index
# get the softmax probabilities
probabilities = torch.nn.functional.softmax(output[0], dim=0)
# check the top 5 categories that are predicted
top5_prob, top5_catid = torch.topk(probabilities, 1)

# Print the predicted class label and inference time
inference_time = end_time - start_time
print(f'Inference time: {inference_time:.4f} seconds')
print(f"Accuracy: {top5_prob[0].item()*100:.3f}%")
print(f"Predicted class:{categories[top5_catid[0]]}")      

During prediction:  2024-10-18 19:07:36.827959+01:00
Inference time: 2.4528 seconds
Accuracy: 99.476%
Predicted class:Newfoundland, Newfoundland dog


In [5]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total trainable parameters: {total_params}')

Total trainable parameters: 25028904


In [6]:
import torch

# Load the pre-trained Mobilenetv3 model
model = resnext50_32x4d(pretrained=True)

# Define input size (224x224x3) for Mobilenetv3
input_size = (224, 224, 3)

# Function to calculate FLOPs for the model
def count_flops(model, input_size):
    # Define a tensor of appropriate size
    input_tensor = torch.randn(1, *input_size)
    
    # Switch to evaluation mode
    model.eval()
    
    # Move model to appropriate device
    device = next(model.parameters()).device
    input_tensor = input_tensor.to(device)
    model.to(device)
    
    # Iterate through model's layers
    flops = 0
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            # For convolutional layers
            output_size = (input_size[0] - module.kernel_size[0] + 2 * module.padding[0]) // module.stride[0] + 1
            output_size = (output_size - module.kernel_size[1] + 2 * module.padding[1]) // module.stride[1] + 1
            flops += module.in_channels * module.out_channels * module.kernel_size[0] * module.kernel_size[1] * output_size * output_size
            input_size = (output_size, output_size, module.out_channels)
        elif isinstance(module, torch.nn.MaxPool2d):
            # For max pooling layers
            output_size = (input_size[0] - module.kernel_size) // module.stride + 1
            flops += input_size[2] * output_size * output_size
            input_size = (output_size, output_size, input_size[2])
        elif isinstance(module, torch.nn.Linear):
            # For fully connected layers
            flops += module.in_features * module.out_features
            input_size = (module.out_features,)
    
    return flops

# Calculate FLOPs
total_flops = count_flops(model, input_size)
print("Total FLOPs:", total_flops)

Total FLOPs: 648726080


Total FLOPs: 648726080
Before loading model:  2024-10-18 19:00:11.910000+01:00

0: 1.0

Loading image:  2024-10-18 19:00:15.327776+01:00
Image transformation:  2024-10-18 19:00:15.685071+01:00
During prediction:  2024-10-18 19:00:22.291703+01:00
Inference time: 2.6147 seconds
Accuracy: 52.723%
Predicted class:toy terrier

datetime.datetime(2024, 10, 18, 19, 00, 11, 156651)
datetime.datetime(2024, 10, 18, 19, 00, 12, 156938)
datetime.datetime(2024, 10, 18, 19, 00, 13, 154920)
datetime.datetime(2024, 10, 18, 19, 00, 14, 164005)
datetime.datetime(2024, 10, 18, 19, 00, 15, 167961)
datetime.datetime(2024, 10, 18, 19, 00, 16, 169016)
datetime.datetime(2024, 10, 18, 19, 00, 17, 161627)
datetime.datetime(2024, 10, 18, 19, 00, 18, 174008)
datetime.datetime(2024, 10, 18, 19, 00, 19, 181167)
datetime.datetime(2024, 10, 18, 19, 00, 20, 175824)
datetime.datetime(2024, 10, 18, 19, 00, 21, 173315)
datetime.datetime(2024, 10, 18, 19, 00, 22, 180897)
datetime.datetime(2024, 10, 18, 19, 00, 23, 224848)
datetime.datetime(2024, 10, 18, 19, 00, 24, 267704)
datetime.datetime(2024, 10, 18, 19, 00, 25, 168767)

Average CPU Temperature: 33.17°C
Average GPU Temperature: 33.83°C
Average Power CPU: 1163.87mW
Average Power GPU: 1163.87mW
Average Power TOT: 2692.20mW
Average RAM: 0.4566

{'Avg_CPU_temp': 33.166666666666664, 'Avg_GPU_temp': 33.833333333333336, 'Avg_Power_CPU': 1163.8666666666666, 'Avg_Power_GPU': 1163.8666666666666, 'Avg_CPU_RAM': 0.45658011516113034}

1: cat_0

Loading image:  2024-10-18 19:01:55.504447+01:00
Image transformation:  2024-10-18 19:01:57.549032+01:00
During prediction:  2024-10-18 19:01:59.868351+01:00
Inference time: 2.5432 seconds
Accuracy: 18.552%
Predicted class:tabby, tabby cat

datetime.datetime(2024, 10, 18, 19, 01, 55, 307682)
datetime.datetime(2024, 10, 18, 19, 01, 56, 313519)
datetime.datetime(2024, 10, 18, 19, 01, 57, 306149)
datetime.datetime(2024, 10, 18, 19, 01, 58, 314732)
datetime.datetime(2024, 10, 18, 19, 01, 59, 308671)
datetime.datetime(2024, 10, 18, 19, 02, 00, 381353)
datetime.datetime(2024, 10, 18, 19, 02, 01, 334899)
datetime.datetime(2024, 10, 18, 19, 02, 02, 365916)

Average CPU Temperature: 33.31°C
Average GPU Temperature: 33.62°C
Average Power CPU: 1382.25mW
Average Power GPU: 1382.25mW
Average Power TOT: 2802.12mW
Average RAM: 0.4632

{'Avg_CPU_temp': 33.3125, 'Avg_GPU_temp': 33.625, 'Avg_Power_CPU': 1382.25, 'Avg_Power_GPU': 1382.25, 'Avg_CPU_RAM': 0.46319655896177614}

2: cat_1

Loading image:  2024-10-18 19:03:00.679943+01:00
Image transformation:  2024-10-18 19:03:03.327356+01:00
During prediction:  2024-10-18 19:03:05.792322+01:00
Inference time: 2.3092 seconds
Accuracy: 80.243%
Predicted class:tabby, tabby cat

datetime.datetime(2024, 10, 18, 19, 03, 00, 395422)
datetime.datetime(2024, 10, 18, 19, 03, 01, 394042)
datetime.datetime(2024, 10, 18, 19, 03, 02, 404790)
datetime.datetime(2024, 10, 18, 19, 03, 03, 392763)
datetime.datetime(2024, 10, 18, 19, 03, 04, 403132)
datetime.datetime(2024, 10, 18, 19, 03, 05, 395147)
datetime.datetime(2024, 10, 18, 19, 03, 06, 431063)
datetime.datetime(2024, 10, 18, 19, 03, 07, 413901)

Average CPU Temperature: 33.12°C
Average GPU Temperature: 33.75°C
Average Power CPU: 1137.88mW
Average Power GPU: 1137.88mW
Average Power TOT: 2589.00mW
Average RAM: 0.4631

{'Avg_CPU_temp': 33.125, 'Avg_GPU_temp': 33.75, 'Avg_Power_CPU': 1137.875, 'Avg_Power_GPU': 1137.875, 'Avg_CPU_RAM': 0.46313694189060023}

3: dog_0

Loading image:  2024-10-18 19:05:04.779654+01:00
Image transformation:  2024-10-18 19:05:07.271733+01:00
During prediction:  2024-10-18 19:05:10.053863+01:00
Inference time: 2.6340 seconds
Accuracy: 98.354%
Predicted class:malamute, malemute, Alaskan malamute

datetime.datetime(2024, 10, 18, 19, 05, 04, 572726)
datetime.datetime(2024, 10, 18, 19, 05, 05, 569109)
datetime.datetime(2024, 10, 18, 19, 05, 06, 573530)
datetime.datetime(2024, 10, 18, 19, 05, 07, 571394)
datetime.datetime(2024, 10, 18, 19, 05, 08, 562855)
datetime.datetime(2024, 10, 18, 19, 05, 09, 574748)
datetime.datetime(2024, 10, 18, 19, 05, 10, 607877)
datetime.datetime(2024, 10, 18, 19, 05, 11, 600752)
datetime.datetime(2024, 10, 18, 19, 05, 12, 639670)

Average CPU Temperature: 33.06°C
Average GPU Temperature: 33.78°C
Average Power CPU: 1248.11mW
Average Power GPU: 1248.11mW
Average Power TOT: 2674.11mW
Average RAM: 0.4631

{'Avg_CPU_temp': 33.05555555555556, 'Avg_GPU_temp': 33.77777777777778, 'Avg_Power_CPU': 1248.111111111111, 'Avg_Power_GPU': 1248.111111111111, 'Avg_CPU_RAM': 0.4630970876869124}

4: cat_2 

Loading image:  2024-10-18 19:06:18.332683+01:00
Image transformation:  2024-10-18 19:06:20.279116+01:00
During prediction:  2024-10-18 19:06:22.978791+01:00
Inference time: 2.2270 seconds
Accuracy: 99.994%
Predicted class:Persian cat

datetime.datetime(2024, 10, 18, 19, 06, 18, 670761)
datetime.datetime(2024, 10, 18, 19, 06, 19, 676472)
datetime.datetime(2024, 10, 18, 19, 06, 20, 674906)
datetime.datetime(2024, 10, 18, 19, 06, 21, 677542)
datetime.datetime(2024, 10, 18, 19, 06, 22, 678873)
datetime.datetime(2024, 10, 18, 19, 06, 23, 697580)
datetime.datetime(2024, 10, 18, 19, 06, 24, 758605)

Average CPU Temperature: 32.86°C
Average GPU Temperature: 34.00°C
Average Power CPU: 1127.43mW
Average Power GPU: 1127.43mW
Average Power TOT: 2542.86mW
Average RAM: 0.4632

{'Avg_CPU_temp': 32.857142857142854, 'Avg_GPU_temp': 34.0, 'Avg_Power_CPU': 1127.4285714285713, 'Avg_Power_GPU': 1127.4285714285713, 'Avg_CPU_RAM': 0.4632361863656392}

5: dog_1

Loading image:  2024-10-18 19:07:29.598854+01:00
Image transformation:  2024-10-18 19:07:32.692963+01:00
During prediction:  2024-10-18 19:07:36.827959+01:00
Inference time: 2.4528 seconds
Accuracy: 99.476%
Predicted class:Newfoundland, Newfoundland dog

datetime.datetime(2024, 10, 18, 19, 07, 29, 762683)
datetime.datetime(2024, 10, 18, 19, 07, 30, 782528)
datetime.datetime(2024, 10, 18, 19, 07, 31, 778781)
datetime.datetime(2024, 10, 18, 19, 07, 32, 767923)
datetime.datetime(2024, 10, 18, 19, 07, 33, 779010)
datetime.datetime(2024, 10, 18, 19, 07, 34, 770226)
datetime.datetime(2024, 10, 18, 19, 07, 35, 782195)
datetime.datetime(2024, 10, 18, 19, 07, 36, 787518)
datetime.datetime(2024, 10, 18, 19, 07, 37, 835858)
datetime.datetime(2024, 10, 18, 19, 07, 38, 836340)

Average CPU Temperature: 32.65°C
Average GPU Temperature: 33.80°C
Average Power CPU: 1005.40mW
Average Power GPU: 1005.40mW
Average Power TOT: 2495.80mW
Average RAM: 0.4632

{'Avg_CPU_temp': 32.65, 'Avg_GPU_temp': 33.8, 'Avg_Power_CPU': 1005.4, 'Avg_Power_GPU': 1005.4, 'Avg_CPU_RAM': 0.4631589164474138}